# 1. Package 설치

In [ ]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone

# 2. Knowledge Base 구성을 위한 데이터 생성

- 4. LangChain을 활용한 Vector Database 변경 (Chroma -> Pinecone)과 동일함

In [20]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# 소득세법
# document_name = './tax_with_markdown.docx'
# 민법
# document_name = './civil_with_markdown.docx'
# 형법
# document_name = './criminal_with_markdown.docx'
# 도로교통법
# document_name = './traffic_with_markdown.docx'
# 근로기준법
document_name = './labor_with_markdown.docx'

loader = Docx2txtLoader(document_name)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [21]:
len_list = len(document_list)
len_list

29

In [4]:
document_list[len_list - 10]

Document(metadata={'source': './traffic_with_markdown.docx'}, page_content='2. 제6조제1항ㆍ제2항ㆍ제4항 또는 제7조에 따른 금지ㆍ제한 또는 조치를 위반한 차 또는 노면전차의 운전자\n\n3. 제22조, 제23조, 제29조제4항부터 제6항까지, 제53조의5, 제60조, 제64조, 제65조 또는 제66조를 위반한 사람\n\n4. 제31조, 제34조 또는 제52조제4항을 위반하거나 제35조제1항에 따른 명령을 위반한 사람\n\n5. 제39조제6항에 따른 시ㆍ도경찰청장의 제한을 위반한 사람\n\n6. 제50조제1항, 제3항 및 제4항을 위반하여 좌석안전띠를 매지 아니하거나 인명보호 장구를 착용하지 아니한 운전자(자전거 운전자는 제외한다)\n\n6의2. 제56조의2제1항을 위반하여 자율주행시스템의 직접 운전 요구에 지체 없이 대응하지 아니한 자율주행자동차의 운전자\n\n7. 제95조제2항에 따른 경찰공무원의 운전면허증 회수를 거부하거나 방해한 사람\n\n8. 삭제<2020. 5. 26.>\n\n9. 삭제<2020. 5. 26.>\n\n9의2. 삭제<2020. 5. 26.>\n\n10. 주ㆍ정차된 차만 손괴한 것이 분명한 경우에 제54조제1항제2호에 따라 피해자에게 인적 사항을 제공하지 아니한 사람\n\n11. 제44조제1항을 위반하여 술에 취한 상태에서 자전거등을 운전한 사람\n\n12. 술에 취한 상태에 있다고 인정할 만한 상당한 이유가 있는 사람으로서 제44조제2항에 따른 경찰공무원의 측정에 응하지 아니한 사람(자전거등을 운전한 사람으로 한정한다)\n\n13. 제43조를 위반하여 제80조에 따른 원동기장치자전거를 운전할 수 있는 운전면허를 받지 아니하거나(원동기장치자전거를 운전할 수 있는 운전면허의 효력이 정지된 경우를 포함한다) 국제운전면허증 또는 상호인정외국면허증 중 원동기장치자전거를 운전할 수 있는 것으로 기재된 국제운전면허증 또는 상호인정외국면허증을 발급받지 아니하고(운전이 금지된 경우와 유효기

In [22]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [23]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

is_create = False
# 소득세법
# index_name = 'tax-markdown-index'
# 민법
# index_name = 'civil-markdown-index'
# 형법
# index_name = 'criminal-markdown-index'
# 도로교통법
# index_name = 'traffic-markdown-index'
# 도로교통법
index_name = 'labor-markdown-index'

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

if is_create:
    # 데이터를 추가할 때
    database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)
else:
    # 이미 생성된 데이터베이스를 사용할 때 
    database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

In [24]:
# 소득세법
# query = '연봉 5천만원인 직장인의 소득세는?'
# 민법
# query = '부동산 매매 계약에서의 소유 이전은 어떻게 이루어지나요?'
# 형법
# query = '소유권 이전 과정에서 허위 서류를 제출한 경우, 이는 어떤 형법적 처벌을 받을 수 있나요?'
# 도로교통법
# query = '음주운전 단속 기준은 어떻게 되나요?'
# 근로기준법
query = '임금 체불 시 어떻게 대응해야 하나요?'

# 3. 답변 생성을 위한 Retrieval

- RetrievalQA에 전달하기 위해 retriever 생성
- search_kwargs의 k값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- .invoke()를 호출해서 어떤 문서를 가져오는지 확인 가능

In [25]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(metadata={'source': './labor_with_markdown.docx'}, page_content='2. 「민사집행법」 제56조제3호에 따른 확정된 지급명령, 하수급인의 근로자에게 하수급인에 대하여 임금채권이 있음을 증명하는 같은 법 제56조제4호에 따른 집행증서,「소액사건심판법」 제5조의7에 따라 확정된 이행권고결정, 그 밖에 이에 준하는 집행권원이 있는 경우\n\n3. 하수급인이 그가 사용한 근로자에 대하여 지급하여야 할 임금채무가 있음을 직상 수급인에게 알려주고, 직상 수급인이 파산 등의 사유로 하수급인이 임금을 지급할 수 없는 명백한 사유가 있다고 인정하는 경우\n\n② 「건설산업기본법」 제2조제10호에 따른 발주자의 수급인(이하 “원수급인”이라 한다)으로부터 공사도급이 2차례 이상 이루어진 경우로서 하수급인(도급받은 하수급인으로부터 재하도급 받은 하수급인을 포함한다. 이하 이 항에서 같다)이 사용한 근로자에게 그 하수급인에 대한 제1항제2호에 따른 집행권원이 있는 경우에는 근로자는 하수급인이 지급하여야 하는 임금(해당 건설공사에서 발생한 임금으로 한정한다)에 해당하는 금액을 원수급인에게 직접 지급할 것을 요구할 수 있다. 원수급인은 근로자가 자신에 대하여 「민법」 제404조에 따른 채권자대위권을 행사할 수 있는 금액의 범위에서 이에 따라야 한다.<개정 2011. 5. 24.>\n\n③ 직상 수급인 또는 원수급인이 제1항 및 제2항에 따라 하수급인이 사용한 근로자에게 임금에 해당하는 금액을 지급한 경우에는 하수급인에 대한 하도급 대금 채무는 그 범위에서 소멸한 것으로 본다.\n\n[본조신설 2007. 7. 27.]\n\n\n\n제45조(비상시 지급) 사용자는 근로자가 출산, 질병, 재해, 그 밖에 대통령령으로 정하는 비상(非常)한 경우의 비용에 충당하기 위하여 임금 지급을 청구하면 지급기일 전이라도 이미 제공한 근로에 대한 임금을 지급하여야 한다.\n\n\n\n제46조(휴업수당) ① 사용자의 귀책사유로 휴업하는 경우에 

# 4. Augmentation을 위한 Prompt 활용

- Retrieval 된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [26]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# 5. 답변 생성

- RetrievalQA를 통해 LLM에 전달
    - RetrievalQA는 create_retrieval_chain으로 대체됨
    - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음
- 하단의 dictionary_chain과 연계하여 사용

In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [28]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [29]:
ai_message = qa_chain.invoke({"query": query})

In [30]:
ai_message

{'query': '임금 체불 시 어떻게 대응해야 하나요?',
 'result': '임금 체불 시, 근로자는 고용노동부에 신고하여 도움을 받을 수 있습니다. 또한, 민사집행법에 따른 확정된 지급명령이나 집행증서를 통해 법적 절차를 밟을 수 있습니다. 체불된 임금은 우선 변제되어야 하며, 체불사업주는 명단 공개 등의 제재를 받을 수 있습니다.'}

# 6. Retrieval을 위한 keyword 사전 활용

- Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정
- LangChain Expression Language (LCEL)을 활용한 Chain 연계

In [31]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 소득세법
# dictionary = ["사람을 나타내는 표현 -> 거주자"]
# 민법
# dictionary = [
#     "사람을 나타내는 표현 -> 개인",
#     "재산의 이전을 나타내는 표현 -> 양도",
#     "결혼을 나타내는 표현 -> 배우자 관계",
#     "법적 책임을 나타내는 표현 -> 책임",
#     "계약의 당사자를 나타내는 표현 -> 계약 당사자"
# ]
# 형법
# dictionary = [
#     "범죄를 나타내는 표현 -> 범죄",
#     "범죄의 피해자를 나타내는 표현 -> 피해자",
#     "형벌을 나타내는 표현 -> 형벌",
#     "공범을 나타내는 표현 -> 공범",
#     "고의성을 나타내는 표현 -> 고의",
#     "과실을 나타내는 표현 -> 과실",
#     "변호권을 나타내는 표현 -> 변호권",
#     "구속을 나타내는 표현 -> 구속",
#     "재판을 나타내는 표현 -> 재판",
#     "형의 집행을 나타내는 표현 -> 형 집행",
#     "보호관찰을 나타내는 표현 -> 보호관찰",
#     "자백을 나타내는 표현 -> 자백",
#     "증거를 나타내는 표현 -> 증거",
#     "형사 고소를 나타내는 표현 -> 고소",
#     "형사 기소를 나타내는 표현 -> 기소",
#     "불법 행위를 나타내는 표현 -> 불법 행위",
#     "유죄를 나타내는 표현 -> 유죄",
#     "무죄를 나타내는 표현 -> 무죄",
#     "혐의를 나타내는 표현 -> 혐의",
#     "집행유예를 나타내는 표현 -> 집행유예"
# ]
# 도로교통법
# dictionary = [
#     "교통법규 위반을 나타내는 표현 -> 교통법규 위반",
#     "교통사고 피해자를 나타내는 표현 -> 피해자",
#     "교통법규 위반에 대한 처벌을 나타내는 표현 -> 처벌",
#     "공동 운전자를 나타내는 표현 -> 공동 운전자",
#     "고의적 교통법규 위반을 나타내는 표현 -> 고의",
#     "과실로 인한 교통법규 위반을 나타내는 표현 -> 과실",
#     "운전자의 변호권을 나타내는 표현 -> 변호권",
#     "운전면허 정지 및 취소를 나타내는 표현 -> 면허 정지/취소",
#     "교통사고 재판을 나타내는 표현 -> 재판",
#     "교통법규 위반에 대한 형 집행을 나타내는 표현 -> 형 집행",
#     "교통사고 피해자에 대한 보호관찰을 나타내는 표현 -> 보호관찰",
#     "운전자의 자백을 나타내는 표현 -> 자백",
#     "교통사고 증거를 나타내는 표현 -> 증거",
#     "교통법규 위반에 대한 고소를 나타내는 표현 -> 고소",
#     "교통법규 위반에 대한 기소를 나타내는 표현 -> 기소",
#     "불법 주/정차를 나타내는 표현 -> 불법 주/정차",
#     "교통법규 위반에 대한 유죄 판결을 나타내는 표현 -> 유죄",
#     "교통법규 위반에 대한 무죄 판결을 나타내는 표현 -> 무죄",
#     "교통사고 혐의를 나타내는 표현 -> 혐의",
#     "교통법규 위반에 대한 집행유예를 나타내는 표현 -> 집행유예"
# ]
# 근로기준법
dictionary = [
    "근로기준법 위반을 나타내는 표현 -> 근로기준법 위반",
    "근로자의 권리를 나타내는 표현 -> 근로자 권리",
    "임금 체불을 나타내는 표현 -> 임금 체불",
    "부당 해고를 나타내는 표현 -> 부당 해고",
    "최저임금을 나타내는 표현 -> 최저임금"
]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}

    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [33]:
new_question = dictionary_chain.invoke({"question": query})

In [34]:
new_question

'임금 체불 시 어떻게 대응해야 하나요?'

In [36]:
law_chain = {"query": dictionary_chain} | qa_chain

In [37]:
ai_response = law_chain.invoke({"question": query})

In [38]:
ai_response

{'query': '임금 체불 시 어떻게 대응해야 하나요?',
 'result': '임금 체불 시, 하수급인은 직상 수급인에게 임금채무를 알리고, 직상 수급인이 임금을 대신 지급할 수 있는 명백한 사유가 있는지 인정받아야 합니다. 또한, 고용노동부에 임금 체불 사실을 신고하고, 필요 시 민사집행법에 따른 지급명령이나 집행증서를 통해 법적 조치를 취할 수 있습니다. 체불사업주의 인적사항이 공개될 수도 있습니다.'}